# Stock Prices Analysis

In this project, we will be dealing with time series data.

## Data Collection

- Numpy
- Pandas
- Matplotlib
- Seaborn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob # to read multiple files

In [9]:
all_files = glob.glob(r'individual_stocks_5yr/*.csv')
print(len(all_files))

505
